In [1]:
!pip install datasets torch_optimizer lion_pytorch clang_repl_kernel --break-system-packages
!pip install --upgrade clang-repl-kernel  --break-system-packages
#!apt update
#!apt-get install libc++1 libc++abi1

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/torchtext-0.18.0a0+9bed85d-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/torchaudio-2.6.0a0+d883142-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg i

In [2]:
from ClangReplInterface import ClangReplInterface
clang_repl = ClangReplInterface()


Lin64.zip: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 350M/350M [00:02<00:00, 118MB/s]


Download completed: /usr/local/lib/python3.12/dist-packages/clang_repl_kernel/clang/Lin64/Lin64.zip
Extracting /usr/local/lib/python3.12/dist-packages/clang_repl_kernel/clang/Lin64/Lin64.zip to /usr/local/lib/python3.12/dist-packages/clang_repl_kernel/clang/Lin64
Reading zip file... Please wait.


Extracting: 100%|████████████████████████████████████████████████████████████████████████████████████| 3245/3245 [00:04<00:00, 681.83file/s]


In [3]:
!rm -rf "runs/starcoder2_reasoning"

In [1]:
import os
import copy
import re
import json
import torch
import gc
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
)
from transformers.optimization import Adafactor
from datasets import Dataset
import signal

# For hyperparameter optimization
import optuna
import pickle
from Config import SimpleConfig
from ClangReplInterface import ClangReplInterface

ref_checkpoint_path = "./saved_models/sample/checkpoint.pt"
last_checkpoint_path = "./saved_models/reasoning/checkpoint.pt"
checkpoint_dir_pre = "./saved_models/reasoning/epoch_"

test_target_object_file = "./saved_models/ReasoningTestTarget_prompt.json"

config = SimpleConfig()

max_length = 512


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:

# num_iterations=1, num_steps=500, batch_size=4, num_generations=4, max_completion_length=128, kl=0.1,
# learning_rate=5e-6, mu=3, epsilon=0.2,
#
# lr: 7.205691481165551e-05 kl_lambda: 0.2654706177039008 epsilon: 0.019437902361559744 num_grpo: 1
# lr: 1.1111588431283189e-06 kl_lambda: 0.15842765249477542 epsilon: 0.11144786260484413 num_grpo: 3

is_finding_opt = False
if not is_finding_opt:
    num_epochs = 200
    lr = 1.1111588431283189e-06
    kl_lambda = 0.15842765249477542
    epsilon = 0.11144786260484413
    num_grpo = 1
    save_epochs = 10
    warming_up_step= 10
    aggregate_gradiation_step = 1


def pad_to_match(tensor_a, tensor_b, padding_value=0):
    # Determine the current sequence lengths
    seq_len_a = tensor_a.size(1)
    seq_len_b = tensor_b.size(1)

    if seq_len_a > seq_len_b:
        max_seq_len = max(seq_len_a, seq_len_b)
    
        # Define padding function
        def pad_tensor(tensor, target_length):
            pad_length = target_length - tensor.size(1)
            if pad_length > 0:
                padding = (0, 0) * (tensor.dim() - 2) + (0, pad_length)
                tensor = F.pad(tensor, padding, value=padding_value)
            return tensor
    
        # Pad both tensors to the maximum sequence length
        tensor_a_padded = pad_tensor(tensor_a, max_seq_len)
        tensor_b_padded = pad_tensor(tensor_b, max_seq_len)
    else:
        tensor_b_padded = tensor_b[:, :seq_len_a]
        tensor_a_padded = tensor_a

    return tensor_a_padded, tensor_b_padded


def reward_atag(front, end, response):
    tag_len = len(front + end)
    start = response.find(front + end)
    end = response.find(front + '/' + end)
    reward = 0
    if start != -1: reward += 0.1
    if end != -1: reward += 0.1
    
    if start + tag_len < end:
        if len(response[start + tag_len:end].strip()) > 1:
            reward += 0.1
    return reward


def remove_comments(code: str):
    pattern = re.compile(r'//.*?$|/\*.*?\*/', re.DOTALL | re.MULTILINE)
    return re.sub(pattern, '', code)


def find_all_tag_indexes(text, tag):
    """Return a list of starting indexes where the tag occurs in the text."""
    indexes = []
    start = 0
    while True:
        idx = text.find(tag, start)
        if idx == -1:
            break
        indexes.append(idx)
        start = idx + len(tag)
    return indexes


def get_tag_start_end(idx, starts, ends, tag, full_text):
    start = starts[idx]+len(tag)
    end = ends[idx]
    return full_text[start: end].strip()
    

def reward_correct(full_text):
    # handle only first answer
    reward = 0.0
    test_target_open = find_all_tag_indexes(full_text, "<Test Target>")
    test_target_close = find_all_tag_indexes(full_text, "</Test Target>")
    clang_repl_open = find_all_tag_indexes(full_text, "<Clang-repl Test>")
    clang_repl_close = find_all_tag_indexes(full_text, "</Clang-repl Test>")
    if len(test_target_open) == 0 or len(test_target_close) == 0 or len(clang_repl_open) == 0 or len(clang_repl_close) == 0:
        return reward, '<Test Target> or <Clang-repl Test> not found'
    if len(test_target_open) != len(test_target_close) or len(clang_repl_open) != len(clang_repl_close):
        return reward , '<Test Target> or <Clang-repl Test> pair not match'
    if not all(x < y for x, y in zip(test_target_open, test_target_close)):
        return reward, '<Test Target> not closed properly'
    if not all(x < y for x, y in zip(clang_repl_open, clang_repl_close)):
        return reward, '<Clang-repl Test> not closed properly' 
    target_text = get_tag_start_end(-1, test_target_open, test_target_close, "<Test Target>", full_text)
    target_text = remove_comments(target_text)
    target_text = ">>> "+target_text.replace('\n', '')

    for idx in range(len(clang_repl_open)):
        clang_repl_test = get_tag_start_end(idx, clang_repl_open, clang_repl_close, "<Clang-repl Test>", full_text)
        clang_repl = ClangReplInterface()
        test_case_with_target = target_text+'\n'+clang_repl_test
        #print(test_case_with_target)
        result, response = clang_repl.run_verify(test_case_with_target)
        reward = 0.0
        if result == 'ok':
            reward = 2.0
        elif result == 'fail':
            reward = 1.0
        elif result == 'error':
            reward = 0.0
        else:
            assert False
        return reward, response
    else:
        return reward, ''

def _reward(completions, full_text):
    # https://blog.gopenai.com/coding-grpo-from-scratch-a-guide-to-distributed-implementation-with-qwen2-5-1-5b-instruct-59b34227edacabs
    responses = completions
    format_rewards = []
    rewards = []
    responses = []
    for response_full in full_text:
        need_more_test_idx = response_full.find('<Need More Test')
        response = response_full if need_more_test_idx == -1 else response_full[:need_more_test_idx]
        score = 0.0
        score += reward_atag("<", "Test Object>", response) # 0.3
        score += reward_atag("<", "Input Data>", response) # 0.3
        score += reward_atag("<", "Expected Output>", response) # 0.3
        score += reward_atag("<", "Clang-repl Test>", response) # 0.3
        score += reward_atag("[", "REASON]", response) * 2 # 0.3*2
        score += reward_atag("[", "ANSWER]", response) * 2 # 0.3*2
        score = score / 10 # 0.24 *10 = 2.4
        format_rewards.append(score)
        correct_reward, response = reward_correct(response)
        rewards.append(score + correct_reward)
        responses.append(response)

    return rewards, responses

# Custom exception for timeouts.
class TimeoutException(Exception):
    pass

def _handle_timeout(signum, frame):
    raise TimeoutException("Function call timed out.")
    
def reward(completions, full_text, writer=None, global_step=None):
    # Set the signal handler and alarm.
    signal.signal(signal.SIGALRM, _handle_timeout)
    signal.alarm(5)  # Set timeout to 5 seconds.
    try:
        results, responses = _reward(completions, full_text)
    except TimeoutException:
        # Log the timeout event to TensorBoard under a specific tag if a writer is provided.
        if writer is not None:
            writer.add_text("TimeoutEvents/RewardFunction", f"Timeout occurred at global step {global_step}", global_step=global_step)
        results = [0]*len(full_text)
        responses = ["reward() timeout"]*len(full_text)
    finally:
        # Cancel the alarm.
        signal.alarm(0)
    return results, responses

def object_hiper_param(trial):
    # Shortened training for demonstration:
    num_epochs = 1  # 2   # or 2–3, to save time during hyperparameter search

    # Sample hyperparameters
    lr = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
    kl_lambda = trial.suggest_float("kl_lambda", 0.0, 1.0)
    epsilon = trial.suggest_float("epsilon", 0.01, 0.2)
    num_grpo = trial.suggest_int("num_grpo", 1, 3, step=1)
    warming_up_step= trial.suggest_int("warming_up_step", 2, 20, step=1)
    aggregate_gradiation_step = trial.suggest_int("aggregate_gradiation_step", 1, 1, step=1)

    return num_epochs, lr, kl_lambda, epsilon, num_grpo, warming_up_step, aggregate_gradiation_step


def print_memory(tag):
    # Make sure you have a GPU device available.
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Print current allocated and reserved memory in MB:
    allocated = torch.cuda.memory_allocated(device) / (1024 ** 2)
    reserved = torch.cuda.memory_reserved(device) / (1024 ** 2)
    print(tag)
    print(f"Memory allocated: {allocated:.2f} MB")
    print(f"Memory reserved: {reserved:.2f} MB")


def samping(model, tokenizer, device, epoch, writer, sample_prompt, expected):
    # Include attention_mask in the tokenization
    sample_prompt = f"### Instruction\n\n{sample_prompt}\n\n### Response"
    inputs = tokenizer(sample_prompt, return_tensors="pt", return_attention_mask=True)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # Pass the attention_mask and explicitly set pad_token_id to eos_token_id for reliable generation
    generated_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=20,
        pad_token_id=tokenizer.eos_token_id
    )
    sample_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    sample_text = sample_text.strip()
    print(f"Sample Output (Epoch {epoch + 1}): {sample_text}")
    print("Expected:", expected)
    writer.add_text("Sample Output", f"Epoch {epoch + 1}: {sample_text}", epoch)



def selective_log_softmax(logits, input_ids, tokenizer):
    # Ensure input_ids are on the same device as logits
    input_ids = input_ids.to(logits.device)

    log_probs = nn.functional.log_softmax(logits, dim=-1)
    if input_ids.size(1) > log_probs.size(1):
        input_ids = input_ids[:, :log_probs.size(1)]

    # Gather log probabilities corresponding to input_ids
    selected_log_probs = log_probs.gather(dim=-1, index=input_ids.unsqueeze(-1)).squeeze(-1)

    if True:
        input_text = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
        print("Input Texts:")
        for text in input_text:
            print(text)
        logits_ids = logits.argmax(dim=-1)
        logit_text = tokenizer.batch_decode(logits_ids, skip_special_tokens=True)
        print("\nLogit Texts:")
        for text in logit_text:
            print(text)

    return selected_log_probs

def add_front_transformer_block(self, copy_weights: bool = True):
    # Retrieve the current first transformer block.
    layer_index = 1
    original_first_block = self.model.layers[layer_index]

    # Create a new block.
    new_block = copy.deepcopy(original_first_block) if copy_weights else type(original_first_block)()

    self.model.layers.insert(layer_index, new_block)

    self.config.num_hidden_layers += 1


def get_layer_params(self, layer_index: int = 0):
    first_params = list(self.model.layers[0].parameters())
    sec_params = list(self.model.layers[0].parameters())
    # last_params = list(self.model.layers[-1].parameters())
    return first_params + sec_params  # + last_params


def gen_logits(model, batch, tokenizer, group_size):
    # Extract input_ids and attention_mask from the batch
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    prompt_length = input_ids.shape[1]
 
    # Generate new tokens.
    output = model.generate(
        input_ids,
        max_length=512,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        output_scores=True,
        return_dict_in_generate=True
    )
    
    # Extract generated token IDs and logits
    generated_ids = output.sequences

    # Separate completion tokens and logits
    completion_ids = generated_ids[:, prompt_length:]

    generated_ids_mask = (generated_ids != tokenizer.pad_token_id).long().to(generated_ids.device)
    logits = model(input_ids=generated_ids, attention_mask=generated_ids_mask).logits

    completion_logits = logits[:, prompt_length-1:, :]

    return completion_logits, completion_ids, generated_ids


# ------------------------------------------------
# Load Q&A from JSON file (manual_data_set/QA.json)
# and create a list of {"content": "..."}
# ------------------------------------------------
def load_qa_dataset(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    train_examples = []
    for item in data:
        q = item.get("Q", "")
        a = item.get("A", "")
        content = f"### Instruction\n\n{q}\n### Response\n\n{a}\n"
        train_examples.append({"content": content + "<|endoftext|>"})
    return train_examples


def load_sample_dataset(pk_file):
    with open(config.dataset_file, "rb") as f:
        global_samples = pickle.load(f)
        sample_dataset = []
        for sample in global_samples:
            sample_dataset.append({"content": sample + "<|endoftext|>"})
        return sample_dataset

def get_test_target_content(full_text):
    test_target_open = find_all_tag_indexes(full_text, "<Test Target>")
    test_target_close = find_all_tag_indexes(full_text, "</Test Target>")
    target_text = get_tag_start_end(-1, test_target_open, test_target_close, "<Test Target>", full_text)
    return target_text

def load_reasoning_dataset(test_target_object_file):
    with open(test_target_object_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
        train = []
        val = []
        categories = set()
        data_dic = {}
        for item in data:
            categories.add(item['category'])
        for cat in categories:
            data_dic[cat] = []
        for item in data:
            data_dic[item['category']].append(item['content'])
        for cat in categories:
            for idx, item in enumerate(data_dic[cat]):
                content = f"### Instruction\n\nn<Test Target>\n{get_test_target_content(item)}\n</Test Target>\nWrtie a Clang-repl Test\n### Response\n"
                if idx >=10:
                    val.append({"content":content})
                else:
                    train.append({"content":content})

        return train, val

# Provide the path to your Q&A JSON file
qa_json_path = "manual_data_set/QA.json"
train_data_prompt = load_qa_dataset(qa_json_path)
train_data_sample = load_sample_dataset(config.dataset_file)
train_data = train_data_prompt + (train_data_sample * 10)

test_target_object_file = "manual_data_set/ReasoningTestTarget.json"
reasoning_dataset, val_reasoning_dataset = load_reasoning_dataset(test_target_object_file)

# Create a Hugging Face Dataset from the list
train_dataset = Dataset.from_list(reasoning_dataset)

# ------------------------------------------------
# Define Tokenization
# ------------------------------------------------
model_id = "bigcode/starcoder2-3b"
# Load tokenizer from saved directory if exists; otherwise, load from pretrained.
tokenizer_save_dir = "./saved_models/tokenizer"
if os.path.exists(tokenizer_save_dir):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_dir)
    print("Loaded tokenizer from saved checkpoint.")
else:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(
        examples["content"],
        truncation=True,
        max_length=512,
        #padding="max_length"
    )

print("eos: ", tokenizer.eos_token, tokenizer.eos_token_id)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["content"])
tokenized_dataset.set_format("torch")

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


# ------------------------------------------------
# Define Training Function
# ------------------------------------------------
def train_and_evaluate(
        model,
        ref_model,
        dataloader,
        optimizer,
        device,
        num_epochs,
        num_grpo,
        epsilon,
        kl_lambda,
        scaler,
        save_epochs,
        start_epoch,
        warming_up_step,
        aggregate_gradiation_step
):
    # Initialize TensorBoard writer (optional)
    writer = SummaryWriter(log_dir="runs/starcoder2_reasoning")

    # --- Generate sample output text after each epoch ---
    model.eval()  # Switch to eval mode for generation
    with torch.no_grad():
        samping(model, tokenizer, device, 0, writer, "In Custom Clang-repl, What is the prompt in Custom Clang-repl?",
                "```\n>>> (prompt)\n```")
        samping(model, tokenizer, device, 0, writer,
                "In Custom Clang-repl, Do we allow multiline comments or backslash-extended lines in Custom Clang-repl Test?",
                "Custom Clang-repl takes only one line input.")
    model.train()  # Switch back to training mode
    group_size=3
    global_step = 0
    for epoch in range(start_epoch, num_epochs):
        running_loss = 0.0
        
        print_memory(20)
        old_model = None
        old_model = copy.deepcopy(model)
        old_model = old_model.half()
        old_model.eval()
        for param in old_model.parameters():
            param.requires_grad = False
        print_memory(21)

        for grpo_idx in range(num_grpo):
            for step, batch in enumerate(dataloader):
                batch = {k: v.to(device) for k, v in batch.items()}
                # Extract input_ids and attention_mask from the batch
                input_ids = batch['input_ids'].repeat_interleave(group_size, dim=0)
                attention_mask = batch['attention_mask'].repeat_interleave(group_size, dim=0)
                
                # Update the batch with the expanded tensors
                batch['input_ids'] = input_ids
                batch['attention_mask'] = attention_mask
                optimizer.zero_grad()

                with torch.cuda.amp.autocast():
                    print("1. Model train")
                    completion_logits, completion_ids, full_ids = gen_logits(model, batch, tokenizer, group_size=group_size)

                    print("2. Reward calc")
                    # Decode the token ids to text strings.
                    output_text_lists = tokenizer.batch_decode(completion_ids, skip_special_tokens=True)
                    full_text_lists = tokenizer.batch_decode(full_ids, skip_special_tokens=True)
                    rewards, responses = reward(output_text_lists, full_text_lists, writer, global_step)
                    advantages = torch.tensor(
                        rewards,
                        dtype=torch.float32,
                        device=device
                    )

                    mean_rewards = advantages.mean()  # .repeat_interleave(num_generations)
                    std_rewards = advantages.std()  # .repeat_interleave(num_generations)

                    _A_hat = (advantages - mean_rewards) / std_rewards
                    A_hat = _A_hat.unsqueeze(1)

                    print("3. Legacy Models Run")
                    # old model forward
                    with torch.no_grad():
                        old_completion_logits, old_completion_ids, old_full_id = gen_logits(old_model, batch, tokenizer, group_size=group_size)

                    # reference model forward
                    with torch.no_grad():
                        ref_completion_logits, ref_completion_ids, ref_full_id = gen_logits(ref_model, batch, tokenizer, group_size=group_size)

                    input_ids = batch["input_ids"]

                    print("4. Loss Calc")
                    # Probability ratio
                    model_log_logits = selective_log_softmax(completion_logits, completion_ids, tokenizer)
                    old_model_log_logits = selective_log_softmax(old_completion_logits, old_completion_ids, tokenizer)
                    model_log_logits, old_model_log_logits = pad_to_match(model_log_logits, old_model_log_logits)
                    probability_ratio = torch.exp(model_log_logits - old_model_log_logits)

                    # Unclipped objective
                    unclipped_objective = probability_ratio * A_hat

                    # Clipped objective
                    clipped_ratio = torch.clamp(probability_ratio, 1 - epsilon, 1 + epsilon)
                    clipped_objective = clipped_ratio * A_hat

                    # ppo_loss = clipped_objective.mean()
                    ppo_loss = -torch.min(unclipped_objective, clipped_objective).mean()
                    # ppo_loss = loss.mean()

                    # KL
                    model_log_probs = F.log_softmax(completion_logits, dim=-1)
                    ref_log_probs = F.log_softmax(ref_completion_logits, dim=-1)
                    ref_log_probs, model_log_probs = pad_to_match(ref_log_probs, model_log_probs)
                    kl_div = F.kl_div(model_log_probs, ref_log_probs, reduction='batchmean')

                    combined_loss = ppo_loss + kl_lambda * kl_div

                print("5. Optimize and update")
                scaler.scale(combined_loss).backward()
                scaler.step(optimizer)
                scaler.update()

                running_loss += combined_loss.item()

                print("6. Logging")
                # TensorBoard logging
                writer.add_scalar("Loss/combined_loss", combined_loss.item(), global_step)
                writer.add_scalar("Loss/ppo_loss", ppo_loss.item(), global_step)
                writer.add_scalar("Loss/kl_div", kl_div.item(), global_step)
                #writer.add_scalar("Loss/original_loss", loss.item(), global_step)

                global_step += 1

        print("7. End Epoch")
        avg_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch + 1} completed. Average Loss: {avg_loss:.4f}")
        writer.add_scalar("Epoch/Average_Loss", avg_loss, epoch + 1)

        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch + 1
        }
        torch.save(checkpoint, last_checkpoint_path)

        if save_epochs is not None and epoch % save_epochs == 0:
            global checkpoint_dir_pre
            checkpoint_dir = checkpoint_dir_pre + str(epoch + 1)
            os.makedirs(checkpoint_dir, exist_ok=True)
            checkpoint = {
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch + 1
            }
            checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.pt")
            torch.save(checkpoint, checkpoint_path)
            print(f"Checkpoint saved at epoch {epoch + 1} to {checkpoint_path}")

            # Save tokenizer only if it has not been saved before.
            tokenizer_save_dir = "./saved_models/tokenizer"
            if not os.path.exists(tokenizer_save_dir):
                os.makedirs(tokenizer_save_dir, exist_ok=True)
                tokenizer.save_pretrained(tokenizer_save_dir)
                print("Tokenizer saved.")

            # --- Generate sample output text after each epoch ---
            model.eval()  # Switch to eval mode for generation
            with torch.no_grad():
                samping(model, tokenizer, device, epoch, writer,
                        "In Custom Clang-repl, What is the prompt in Custom Clang-repl?", "```\n>>> (prompt)\n```")
                samping(model, tokenizer, device, epoch, writer,
                        "In Custom Clang-repl, Do we allow multiline comments or backslash-extended lines in Custom Clang-repl Test?",
                        "Custom Clang-repl takes only one line input.")
                samping(model, tokenizer, device, epoch, writer, "Make python string reverse function",
                        "def reverse(text):\n    return reverse(text[1:])+text[0]")
                samping(model, tokenizer, device, epoch, writer,
                        "<Test Target Object>\nAdd two integers. and return the sum.\n</Test Target Object>\n", "....")
                samping(model, tokenizer, device, epoch, writer,
                        "<Test Target>\nbool isEven(int x) {\n    return (x % 2) == 0;\n}\n</Test Target>\n", "....")
                print(
                    "=====================================================================================================")
            model.train()  # Switch back to training mode

    writer.close()

    # Return final average loss as the metric to minimize
    return avg_loss


def train(
        num_epochs,
        lr,
        kl_lambda,
        epsilon,
        num_grpo,
        warming_up_step,
        aggregate_gradiation_step,
        save_epochs=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Check if a latest checkpoint exists to load model and optimizer states
    if os.path.exists(last_checkpoint_path):
        checkpoint = torch.load(last_checkpoint_path, map_location=torch.device("cpu"))
        _model = AutoModelForCausalLM.from_pretrained(model_id)
        config = _model.config
        config.num_hidden_layers += 2
        model = AutoModelForCausalLM.from_config(config)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
        optimizer = Adafactor(get_layer_params(model), lr=lr, relative_step=False, scale_parameter=False)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint.get('epoch', 0)
        print(f"Loaded checkpoint from {last_checkpoint_path} at epoch {start_epoch}")
    else:
        if os.path.exists(ref_checkpoint_path):
            print_memory(1)
            checkpoint = torch.load(ref_checkpoint_path, map_location=torch.device("cpu"))
            _model = AutoModelForCausalLM.from_pretrained(model_id)
            config = copy.deepcopy(_model.config)
            _model = None
            print_memory(2)
            config.num_hidden_layers += 2
            model = AutoModelForCausalLM.from_config(config)
            print_memory(3)
            model.load_state_dict(checkpoint['model_state_dict'])
            model.to(device)
            print_memory(4)
            optimizer = Adafactor(get_layer_params(model), lr=lr, relative_step=False, scale_parameter=False)
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = 0 #checkpoint.get('epoch', 0)
            print(f"Loaded checkpoint from {ref_checkpoint_path} at epoch {start_epoch}")
            print_memory(7)

        else:
            assert (False, "prompt_last_checkpoint_path must exist")

    # Clear cached memory that is no longer used
    torch.cuda.empty_cache()
    gc.collect()
    print_memory(9)

    # Reference model (for KL)
    old_model = None
    ref_model = copy.deepcopy(model).half().eval()
    for param in ref_model.parameters():
        param.requires_grad = False
    print_memory(10)
    
    # DataLoader
    batch_size = 1
    dataloader = DataLoader(
        tokenized_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator
    )

    # AMP GradScaler
    scaler = torch.cuda.amp.GradScaler()

    # Train & get final metric
    final_avg_loss = train_and_evaluate(
        model=model,
        ref_model=ref_model,
        dataloader=dataloader,
        optimizer=optimizer,
        device=device,
        num_epochs=num_epochs,
        num_grpo=num_grpo,
        epsilon=epsilon,
        kl_lambda=kl_lambda,
        scaler=scaler,
        save_epochs=save_epochs,
        start_epoch=start_epoch,
        warming_up_step=warming_up_step,
        aggregate_gradiation_step=aggregate_gradiation_step
        
    )

    # Return the final average loss to Optuna
    return final_avg_loss


# ------------------------------------------------
# Optuna Objective Function
# ------------------------------------------------
def objective(trial):
    """
    Defines how Optuna will run each trial:
    - sample hyperparameters
    - set up the model & optimizer with those
    - run a short training loop
    - return a metric (the final avg loss) to minimize
    """
    num_epochs, lr, kl_lambda, epsilon, num_grpo, warming_up_step, aggregate_gradiation_step = object_hiper_param(trial)

    print(
        f"[Optuna] Trial hyperparameters -> lr: {lr}, kl_lambda: {kl_lambda}, epsilon: {epsilon}, num_grpo: {num_grpo}")
    return train(
        num_epochs=num_epochs,
        lr=lr,
        kl_lambda=kl_lambda,
        epsilon=epsilon,
        num_grpo=num_grpo,
        warming_up_step=warming_up_step, 
        aggregate_gradiation_step=aggregate_gradiation_step
    )


# ------------------------------------------------
# Run Optuna Study
# ------------------------------------------------
if __name__ == "__main__":
    if is_finding_opt:
        # Create study to minimize final loss
        study = optuna.create_study(direction="minimize")
        study.optimize(objective, n_trials=5)  # You can increase n_trials

        print("Study completed!")
        print("Best trial:")
        best_trial = study.best_trial
        print(f"  Value: {best_trial.value}")
        print("  Params: ")
        for key, value in best_trial.params.items():
            print(f"#    {key}: {value}")
        # Study completed!
        # Best trial:
        #  Value: 715.3611988491482
        #  Params:
        #    lr: 0.0002746775018590349
        #    kl_lambda: 0.10527608699361579
        #    epsilon: 0.12442505216944565
        #    num_grpo: 2
    else:
        train(
            num_epochs=num_epochs,
            lr=lr,
            kl_lambda=kl_lambda,
            epsilon=epsilon,
            num_grpo=num_grpo,
            save_epochs=save_epochs,
            warming_up_step=warming_up_step,
            aggregate_gradiation_step=aggregate_gradiation_step
        )


<>:626: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:626: SyntaxWarning: assertion is always true, perhaps remove parentheses?


Loaded tokenizer from saved checkpoint.
eos:  <|endoftext|> 0


Map:   0%|          | 0/90 [00:00<?, ? examples/s]

1
Memory allocated: 0.00 MB
Memory reserved: 0.00 MB


/tmp/ipykernel_1648/3122056573.py:626: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (False, "prompt_last_checkpoint_path must exist")


...

10
Memory allocated: 18449.15 MB
Memory reserved: 24930.00 MB


/tmp/ipykernel_1648/3122056573.py:650: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Sample Output (Epoch 1): ### Instruction

In Custom Clang-repl, What is the prompt in Custom Clang-repl?

### Response

```
>>> (prompt)
```

### Response

```
>>> (
Expected: ```
>>> (prompt)
```


/tmp/ipykernel_1648/3122056573.py:451: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Sample Output (Epoch 1): ### Instruction

In Custom Clang-repl, Do we allow multiline comments or backslash-extended lines in Custom Clang-repl Test?

### Response

Custom Clang-repl takes only one line input.
### Response

Custom C
Expected: Custom Clang-repl takes only one line input.
20
Memory allocated: 18457.27 MB
Memory reserved: 24934.00 MB
21
Memory allocated: 24606.25 MB
Memory reserved: 31062.00 MB
1. Model train
2. Reward calc
3. Legacy Models Run
4. Loss Calc
Input Texts:

[REASON]
<Test Target>
void sort(int arr[], int n) {
for (int i = 0; i < n - 1; i++) {
for (int j = i; j < n - i - 1; j++) {
if (arr[j] > arr[j + 1]) {
int temp = arr[j];
arr[j] = arr[j + 1];
arr[j + 1] = temp;
}
}
}
</Test Target>

<Test Object>
Verify that sort correctly places elements in Reverse Order when elements are increasing Order.
</Test Object>
<Input Data>
>>> int arr[6] = {1, 2, 3, 4, 5, 6};
>>> int n = 6;
</Input Data>
<Expected Output>
>>> %<< sorted_arr == {1, 2, 3, 4, 5, 6};
true
</Expect

In [ ]:
A_hat.unsqueeze(1)